# Document Similarity Graph Generation

## TODO
 * Load documents in topic
 * Compute tf-idf representation of topic
 * Compute pairwise similarities
 * Build graph based on similarities
 * Output graph in dot format and visualize with graphviz
 * Clean up the notebook (will look like crap at this point)
 * Explore using simhashing to speed up the similarity lookups

In [1]:
import numpy as np
from sklearn.feature_extraction.text import *